In [43]:
# class for the dataset:
import torch
# class for text-datasets
# Jama, NEJM and lancet
from torch.utils.data import Dataset, DataLoader

class TextDataset(Dataset):

    def __len__(self):
        return len(self.total_examples)
        pass
 
    # generates tensors for the vocab
    # i.e. assigns a number to each character
    # Q: possible to do one-pass?
    def make_vocab(self):
        
        self.char2index = {}
        self.index2char = {}
        index = 0
        with open(self.text_file) as file:
            for line in file:
                for char in line:
                    if char not in self.char2index:
                        self.char2index[char] = index
                        self.index2char[index]  = char
                        index+=1
        
        # potentially need to add 2 for the None and \n characters
        self.vocab_size = len(self.char2index)
    
    def generate_tensor_for_char(self, char):
        temp = torch.zeros(1, self.vocab_size)
        temp[0][self.char2index[char]] = 1 
        return temp

#     def generate_char_from_tensor(self, char):
#         temp = torch.zeros(self.vocab_size, 1)
#         temp[self.char2index[char]][0] = 1 
#         return temp
    def lineToTensor(self,line):
        tensor = torch.zeros(len(line), 1, self.vocab_size)
        for li, letter in enumerate(line):
            tensor[li][0][self.char2index[letter]] = 1
        return tensor
    
    
    def __init__(self, string, text_file):
        self.string = string
        self.text_file = text_file

        self.training_examples = []

        total_inputs = []
        total_outputs = []
        
#         make the vocab
        self.make_vocab()
        
        with open(text_file) as file:
            for line in file:
                # zero pad to start
                inputs = [None] + [x for x in line]
                targets = inputs[1:] + ["\n"]  # we need a 0 as well!

                # append: will be a list of list (of lines)
                # extend: will be a list of lines
                total_inputs.append(inputs)
                total_outputs.append(targets)

        assert len(total_inputs) == len(total_outputs)
        
        # this is now a LIST of lists!
        self.total_examples = list(zip(total_inputs, total_outputs))
        
        #                 for char in line:
        # how many chars before the target? or all chars before the target?
        # so we simply pair up x with [x+1] i think
        # ok, and then we iterate them in the loader?
        #                     inputs = [x for x in ]
        #                     targets = []

        # we should also have some big list of entries 

        pass
    
    
    # ok, now let us just try adding the transforms
    # actually the transforms should take responsibility for all of the vocab stuff
    
    # number of items in the dataset. This involves a rather large computation!
    # we can either do a feed forward cllm in the style of bengio, or we can just 
    # do the RNN approach (as specified by Sean Robertson in his tutorial)

    # the indices should represent examples inside the list
    # one strange thing is the following: we need to precompute al the indices earlier, imo, and then simply return the list
    # at that element! (perhaps with some transforms!)
    def __getitem__(self, index):
        return self.total_examples[index]

        # build a list, return elements from that list? we need to internally keep track of some index
        pass



# ok, so it works now!

In [40]:
abc = TextDataset("rand", "data\\names\\English.txt")


In [41]:
abc.generate_tensor_for_char("a")
# abc.index2char[2]
abc.lineToTensor("abcd")
    

tensor([[[0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0.]],

        [[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
 

In [38]:


for (a,b) in abc:
#     this should give two lines/tests
    print(a,b)
    
#     realistically, we should also have it as tensors/vectors
# finally, we should also have some way where we run the rnn forward and 

[None, 'A', 'b', 'b', 'a', 's', '\n'] ['A', 'b', 'b', 'a', 's', '\n', '\n']
[None, 'A', 'b', 'b', 'e', 'y', '\n'] ['A', 'b', 'b', 'e', 'y', '\n', '\n']
[None, 'A', 'b', 'b', 'o', 't', 't', '\n'] ['A', 'b', 'b', 'o', 't', 't', '\n', '\n']
[None, 'A', 'b', 'd', 'i', '\n'] ['A', 'b', 'd', 'i', '\n', '\n']
[None, 'A', 'b', 'e', 'l', '\n'] ['A', 'b', 'e', 'l', '\n', '\n']
[None, 'A', 'b', 'r', 'a', 'h', 'a', 'm', '\n'] ['A', 'b', 'r', 'a', 'h', 'a', 'm', '\n', '\n']
[None, 'A', 'b', 'r', 'a', 'h', 'a', 'm', 's', '\n'] ['A', 'b', 'r', 'a', 'h', 'a', 'm', 's', '\n', '\n']
[None, 'A', 'b', 'r', 'a', 'm', 's', '\n'] ['A', 'b', 'r', 'a', 'm', 's', '\n', '\n']
[None, 'A', 'c', 'k', 'a', 'r', 'y', '\n'] ['A', 'c', 'k', 'a', 'r', 'y', '\n', '\n']
[None, 'A', 'c', 'k', 'r', 'o', 'y', 'd', '\n'] ['A', 'c', 'k', 'r', 'o', 'y', 'd', '\n', '\n']
[None, 'A', 'c', 't', 'o', 'n', '\n'] ['A', 'c', 't', 'o', 'n', '\n', '\n']
[None, 'A', 'd', 'a', 'i', 'r', '\n'] ['A', 'd', 'a', 'i', 'r', '\n', '\n']
[None, '

[None, 'D', 'e', 'l', 'l', '\n'] ['D', 'e', 'l', 'l', '\n', '\n']
[None, 'D', 'e', 'm', 'p', 's', 'e', 'y', '\n'] ['D', 'e', 'm', 'p', 's', 'e', 'y', '\n', '\n']
[None, 'D', 'e', 'm', 'p', 's', 't', 'e', 'r', '\n'] ['D', 'e', 'm', 'p', 's', 't', 'e', 'r', '\n', '\n']
[None, 'D', 'e', 'n', 'b', 'y', '\n'] ['D', 'e', 'n', 'b', 'y', '\n', '\n']
[None, 'D', 'e', 'n', 'h', 'a', 'm', '\n'] ['D', 'e', 'n', 'h', 'a', 'm', '\n', '\n']
[None, 'D', 'e', 'n', 'i', 's', '\n'] ['D', 'e', 'n', 'i', 's', '\n', '\n']
[None, 'D', 'e', 'n', 'n', 'e', 'y', '\n'] ['D', 'e', 'n', 'n', 'e', 'y', '\n', '\n']
[None, 'D', 'e', 'n', 'n', 'i', 's', '\n'] ['D', 'e', 'n', 'n', 'i', 's', '\n', '\n']
[None, 'D', 'e', 'n', 't', '\n'] ['D', 'e', 'n', 't', '\n', '\n']
[None, 'D', 'e', 'n', 't', 'o', 'n', '\n'] ['D', 'e', 'n', 't', 'o', 'n', '\n', '\n']
[None, 'D', 'e', 'p', 'p', '\n'] ['D', 'e', 'p', 'p', '\n', '\n']
[None, 'D', 'e', 'r', 'm', 'o', 'd', 'y', '\n'] ['D', 'e', 'r', 'm', 'o', 'd', 'y', '\n', '\n']
[None, '

[None, 'G', 'o', 'l', 'd', 'b', 'e', 'r', 'g', '\n'] ['G', 'o', 'l', 'd', 'b', 'e', 'r', 'g', '\n', '\n']
[None, 'G', 'o', 'l', 'd', 'i', 'n', 'g', '\n'] ['G', 'o', 'l', 'd', 'i', 'n', 'g', '\n', '\n']
[None, 'G', 'o', 'l', 'd', 'm', 'a', 'n', '\n'] ['G', 'o', 'l', 'd', 'm', 'a', 'n', '\n', '\n']
[None, 'G', 'o', 'l', 'd', 's', 'm', 'i', 't', 'h', '\n'] ['G', 'o', 'l', 'd', 's', 'm', 'i', 't', 'h', '\n', '\n']
[None, 'G', 'o', 'l', 'd', 's', 'w', 'o', 'r', 't', 'h', 'y', '\n'] ['G', 'o', 'l', 'd', 's', 'w', 'o', 'r', 't', 'h', 'y', '\n', '\n']
[None, 'G', 'o', 'm', 'e', 'z', '\n'] ['G', 'o', 'm', 'e', 'z', '\n', '\n']
[None, 'G', 'o', 'n', 'z', 'a', 'l', 'e', 'z', '\n'] ['G', 'o', 'n', 'z', 'a', 'l', 'e', 'z', '\n', '\n']
[None, 'G', 'o', 'o', 'c', 'h', '\n'] ['G', 'o', 'o', 'c', 'h', '\n', '\n']
[None, 'G', 'o', 'o', 'd', '\n'] ['G', 'o', 'o', 'd', '\n', '\n']
[None, 'G', 'o', 'o', 'd', 'a', 'c', 'r', 'e', '\n'] ['G', 'o', 'o', 'd', 'a', 'c', 'r', 'e', '\n', '\n']
[None, 'G', 'o', 'o'

[None, 'M', 'a', 'j', 'o', 'r', '\n'] ['M', 'a', 'j', 'o', 'r', '\n', '\n']
[None, 'M', 'a', 'k', 'i', 'n', '\n'] ['M', 'a', 'k', 'i', 'n', '\n', '\n']
[None, 'M', 'a', 'l', 'l', 'e', 'y', '\n'] ['M', 'a', 'l', 'l', 'e', 'y', '\n', '\n']
[None, 'M', 'a', 'l', 'l', 'i', 'n', 's', 'o', 'n', '\n'] ['M', 'a', 'l', 'l', 'i', 'n', 's', 'o', 'n', '\n', '\n']
[None, 'M', 'a', 'l', 'o', 'n', 'e', '\n'] ['M', 'a', 'l', 'o', 'n', 'e', '\n', '\n']
[None, 'M', 'a', 'l', 'o', 'n', 'e', 'y', '\n'] ['M', 'a', 'l', 'o', 'n', 'e', 'y', '\n', '\n']
[None, 'M', 'a', 'n', 'g', 'n', 'a', 'l', 'l', '\n'] ['M', 'a', 'n', 'g', 'n', 'a', 'l', 'l', '\n', '\n']
[None, 'M', 'a', 'n', 'n', '\n'] ['M', 'a', 'n', 'n', '\n', '\n']
[None, 'M', 'a', 'n', 'n', 'i', 'n', 'g', '\n'] ['M', 'a', 'n', 'n', 'i', 'n', 'g', '\n', '\n']
[None, 'M', 'a', 'n', 's', 'e', 'l', 'l', '\n'] ['M', 'a', 'n', 's', 'e', 'l', 'l', '\n', '\n']
[None, 'M', 'a', 'n', 's', 'f', 'i', 'e', 'l', 'd', '\n'] ['M', 'a', 'n', 's', 'f', 'i', 'e', 'l', '

[None, 'R', 'o', 'l', 'l', 'i', 'n', 's', 'o', 'n', '\n'] ['R', 'o', 'l', 'l', 'i', 'n', 's', 'o', 'n', '\n', '\n']
[None, 'R', 'o', 'm', 'a', 'n', '\n'] ['R', 'o', 'm', 'a', 'n', '\n', '\n']
[None, 'R', 'o', 'm', 'a', 'n', 's', '\n'] ['R', 'o', 'm', 'a', 'n', 's', '\n', '\n']
[None, 'R', 'o', 'n', 'a', 'l', 'd', '\n'] ['R', 'o', 'n', 'a', 'l', 'd', '\n', '\n']
[None, 'R', 'o', 'n', 'f', 'l', 'a', 'r', 'd', '\n'] ['R', 'o', 'n', 'f', 'l', 'a', 'r', 'd', '\n', '\n']
[None, 'R', 'o', 'o', 'k', '\n'] ['R', 'o', 'o', 'k', '\n', '\n']
[None, 'R', 'o', 'o', 'k', 'e', '\n'] ['R', 'o', 'o', 'k', 'e', '\n', '\n']
[None, 'R', 'o', 'o', 'm', 'e', '\n'] ['R', 'o', 'o', 'm', 'e', '\n', '\n']
[None, 'R', 'o', 'o', 'n', 'e', 'y', '\n'] ['R', 'o', 'o', 'n', 'e', 'y', '\n', '\n']
[None, 'R', 'o', 'o', 't', 'h', 'a', 'm', '\n'] ['R', 'o', 'o', 't', 'h', 'a', 'm', '\n', '\n']
[None, 'R', 'o', 'p', 'e', 'r', '\n'] ['R', 'o', 'p', 'e', 'r', '\n', '\n']
[None, 'R', 'o', 'p', 'p', 'l', 'e', '\n'] ['R', 'o', 

In [ ]:
# ok, so now let's just do the stuff
# the RNN actually makes one-character over predictions! 
# rnn_forward is a utility function that does a lot of stuff

